In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
import datetime as dt
from datetime import timedelta
%matplotlib inline


In [2]:
o_time = 10*3600*1000000 
c_time = 17*3600*1000000 

In [3]:
time_df = pd.DataFrame([o_time,c_time], columns=['adj mtime'])

In [4]:
def metric(filename):
    df = pd.read_csv(filename)
#     print(filename)
#     print(len(df))
    
    if len(df)>0:
        df['venue'] = df['#RIC'].str.split('.', expand=True)[1]
        df['stock'] = df['#RIC'].str.split('.', expand=True)[0]
        
        # df['timestamp'] = (df['Date[L]'] + df['Time[L]'])
        df['timestamp'] = pd.to_datetime(df['Date-Time'], utc = True)
        df['timestamp'] = df['timestamp'].dt.tz_convert('Pacific/Auckland')
        df['date'] = df['timestamp'].dt.date
        df['time'] = df['timestamp'].dt.time
        #df.to_csv('C:/Users/anche/NZ/trth data/tester_out/'+'1_'+file_name)
        #df['adj timestamp'] = df['timestamp']-timedelta(hours=13)
        # df['adj timestamp'] = df['timestamp']-timedelta(hours=13)
        #correct daylight saving 
        #df['adj timestamp'] = np.where(((df['date']>date(2017,10,1))&(df['date']<=date(2018,3,31)))|(df['date']>date(2018,9,30)), df['adj timestamp']+timedelta(hours=1),df['adj timestamp'])
        # df['adj timestamp'] = df['adj timestamp']-timedelta(hours=13)
        #correct time 
        #column went wrong 
        #df[['Price','Volume','Buyer ID','Seller ID','Qualifiers','Tick Dir.']] = df.groupby(['stock','date'])[['Price','Volume','Buyer ID','Seller ID','Qualifiers','Tick Dir.']].shift(-2)
        # df['Price'] = df.groupby(['stock','date'])['Price'].shift(-2)
        # df['Volume'] = df.groupby(['stock','date'])['Volume'].shift(-2)
        # df['Buyer ID'] = df.groupby(['stock','date'])['Buyer ID'].shift(-2)
        # df['Seller ID'] = df.groupby(['stock','date'])['Seller ID'].shift(-2)
        # df['Qualifiers'] = df.groupby(['stock','date'])['Qualifiers'].shift(-2)
        # df['Tick Dir.'] = df.groupby(['stock','date'])['Tick Dir.'].shift(-2)

        # df.to_csv('ARG_v3.csv')
        # df = df[df['Type']=='Quote']
        #df[['Price','Bid Price','Ask Price','Volume','Bid Size','Ask Size']] = pd.to_numeric(df[['Price','Bid Price','Ask Price','Volume','Bid Size','Ask Size']],errors = 'coerce')
        df = df[(df['Price'].notnull())|(df['Bid Price'].notnull())]

        #for realized spread (hypothetical timeframe)
        # df['TS_1m'] = df['adj timestamp'] + timedelta(minutes=1)

        #time in microsecond 
        df['mtime'] = df['time'].apply(lambda x: x.hour * 3600*1000000 + x.minute * 60 *1000000 + x.second *1000000+ x.microsecond)
        df['adj mtime'] = np.where(((df['date']>date(2017,10,1))&(df['date']<=date(2018,3,31)))|(df['date']>date(2018,9,30)), df['mtime']+3600*1000000,df['mtime'])
        #df['adj mtime'] = np.where(df['Type']=='Trade',df['adj mtime']-60 *1000000,df['adj mtime'])
        df['Ask Size abs'] =df['Ask Size'] 
        df['Bid Size abs'] = df['Bid Size'] 
        #df = pd.concat([o_time_df,df.set_index('adj mtime'),c_time_df],axis = 0)
        df['seq'] = df.index
    #     print(df)
        if len(df)>1:
            df = df.merge(time_df, on = ['adj mtime'], how = 'outer')
            df = df.sort_values(['adj mtime','seq'])
            df[['stock','venue','date']] = df[['stock','venue','date']].ffill()
            df['mtime_1m'] = df['adj mtime'] + 60 *1000000
            df['mtime_1s'] = df['adj mtime'] + 1 *1000000
            df['mtime_30s'] = df['adj mtime'] + 30 *1000000
            df['mtime_5m'] = df['adj mtime'] + 5 *60 *1000000
            # df = pd.merge_asof(df,time_df, on = ['adj mtime'], direction = 'backward')
            df['lock'] = np.where(df['Bid Price']==df['Ask Price'],1,0)
            # fil in quoting status of the previous quote
            quotes_cols = ['Bid Price', 'Bid Size', 'Ask Price', 'Ask Size', 'Ask Size abs', 'Bid Size abs']
            # df = df.sort_values(['venue', 'extra_timestamp'])
            df[quotes_cols] = df.groupby(['venue','stock','date'])[quotes_cols].fillna(method='ffill')
            df['move'] = np.where(df['Bid Price']!=df['Bid Price'].shift(1),'B',np.nan)
            df['move'] = np.where(df['Ask Price']!=df['Ask Price'].shift(1),'A',df['move'])

            # df['move'] = df['move'].replace({'nan':np.nan})
            # df['move'] =df['move'].ffill()
            df['quote_direction'] = np.where((df['lock']==1)&(df['move']=='B'),'B',np.nan)
            df['quote_direction'] = np.where((df['lock']==1)&(df['move']=='A'),'S',df['quote_direction'])

            df['direction'] = df['quote_direction'].replace({'nan':np.nan})
            df['direction'] = df.groupby(['venue','stock','date'])['direction'].ffill()
            df['direction'] = np.where(df['Price'].notnull(),df['direction'],np.nan)
            
            df['trad_direction'] = np.where(df['Price']==df['Ask Price'],'B',np.nan)
            df['trad_direction'] = np.where(df['Price']==df['Bid Price'],'S',df['trad_direction'])
            
            df['direction'] = np.where(df['direction'].notnull(),df['direction'],df['trad_direction'])

            # df['Bid Price adj'] = np.where((df['lock']==1)&(df['move']=='B'),np.nan,df['Bid Price'])
            # df['Ask Price adj'] = np.where((df['lock']==1)&(df['move']=='A'),np.nan,df['Ask Price'])

            # df['Bid Price adj'] = np.where((df['lock']==1),np.nan,df['Bid Price'])
            # df['Ask Price adj'] = np.where((df['lock']==1),np.nan,df['Ask Price'])

            # df['Bid Price adj'] = df['Bid Price adj'].ffill()
            # df['Ask Price adj'] = df['Ask Price adj'].ffill()

#             df['Bid Price adj'] = np.where((df['lock']==1)&(df['lock'].shift()==0),df['Bid Price'].shift(1),np.nan)
#             df['Ask Price adj'] = np.where((df['lock']==1)&(df['lock'].shift()==0),df['Ask Price'].shift(1),np.nan)
#             df['Bid Price adj'] = df.groupby(['venue','stock','date'])['Bid Price adj'].ffill()
#             df['Ask Price adj'] = df.groupby(['venue','stock','date'])['Ask Price adj'].ffill()
#             df['Bid Price adj'] = np.where(df['Bid Price adj'].notnull(), df['Bid Price adj'], df['Bid Price'])
#             df['Ask Price adj'] = np.where(df['Ask Price adj'].notnull(), df['Ask Price adj'], df['Ask Price'])
            #df.to_csv('C:/Users/anche/NZ/trth data/tester_out/'+'2_'+file_name)
            # df['new_index'] = df.groupby(['date','stock','venue','lock'])['Bid Price'].cumcount()
            df['Bid Price adj'] = df['Bid Price']
            df['Ask Price adj'] = df['Ask Price']


            df['MidQuote'] = np.where((df['Bid Price'] != 0) & (df['Ask Price'] != 0),
                                              (df['Bid Price'] + df['Ask Price']) / 2.0, np.nan)

            df['MidQuote adj'] = np.where((df['Bid Price adj'] != 0) & (df['Ask Price adj'] != 0),
                                              (df['Bid Price adj'] + df['Ask Price adj']) / 2.0, np.nan)

            # assign trade direction

            # df['direction'] = np.where(df['Price']>df['MidQuote'], 'B',np.nan)
            # df['direction'] = np.where(df['Price']<df['MidQuote'],'S',df['direction'])
            # df['direction'] = np.where(df['Price']==df['MidQuote'],'C',df['direction'])

            # o_time = dt.time(10, 0, 0)
            # c_time = dt.time(17, 0, 0)
            # df = df[(df['time'] > o_time) & (df['time'] < c_time)]

            #whether filter by hours trade
            #df = df[(df['adj mtime'] >= o_time) & (df['adj mtime'] <= c_time)]
            df['NZ Price'] = df['Price']
            df['NZ Bid Price adj'] = df['Bid Price adj']
            df['NZ Ask Price adj'] = df['Ask Price adj']
            df['NZ MidQuote adj'] = df['MidQuote adj']
            # #adjust currency
#             daily_fx = pd.read_csv('C:/Users/anche/NZ/trth data/3 month/daily fx 3 month.csv')
#             daily_fx['date'] = pd.to_datetime(daily_fx['date']).dt.date
#             daily_fx['midquote'] = pd.to_numeric(daily_fx['midquote'], errors = 'coerce')
#             df = df.merge(daily_fx, on = ['date'], suffixes = ['','_fx'], how = 'left')
#             df['Price'] = df['Price']*df['midquote']
#             df['Bid Price'] = df['Bid Price']*df['midquote']
#             df['Ask Price'] = df['Ask Price']*df['midquote']
#             df['MidQuote'] = df['MidQuote']*df['midquote']

#             df['Bid Price adj'] = df['Bid Price adj']*df['midquote']
#             df['Ask Price adj'] = df['Ask Price adj']*df['midquote']
#             df['MidQuote adj'] = df['MidQuote adj']*df['midquote']

            #continue other metric
            df['value'] = df['Price']*df['Volume']

            df['old fee'] = df['value'].apply(lambda x: min(1+0.00002*x, 75)) 
            df['relative old fee'] = (df['old fee']/df['value'])*10000
            df['new fee'] = df['value'].apply(lambda x: min(0.000045*x, 75))
            df['relative new fee'] = (df['new fee']/df['value'])*10000
            relative_old_fee = df.groupby(['date','stock'])['relative old fee'].mean().reset_index()
            relative_new_fee = df.groupby(['date','stock'])['relative new fee'].mean().reset_index()

            total_old_fee = df.groupby(['date','stock'])['old fee'].sum().reset_index()
            total_new_fee = df.groupby(['date','stock'])['new fee'].sum().reset_index()

            df['fee diff'] = df['new fee'] - df['old fee']
            df['relative fee diff'] = df['fee diff']/df['value']
            df['fee change'] = df['fee diff']/((df['new fee'] + df['old fee'])/2)
            #per stock per day fee difference
            fee_diff = df.groupby(['date','stock'])['fee diff'].sum().reset_index()
            per_stock_fee_diff = fee_diff.groupby(['stock'])['fee diff'].mean().reset_index()

            relative_fee_diff = df.groupby(['date','stock'])['relative fee diff'].mean().reset_index()

            fee_change = df.groupby(['date','stock'])['fee change'].mean().reset_index()
            per_stock_fee_change = fee_change.groupby(['stock'])['fee change'].mean().reset_index()

            df['fee increase'] = np.where(df['new fee']>df['old fee'],1,0)
            df['fee decrease'] = np.where(df['new fee']<df['old fee'],1,0)
            df['fee unchanged'] = np.where(df['new fee']==df['old fee'],1,0)

            df['trades'] = np.where(df['Price'].notnull(),1,0)
            fee_increase = (df.groupby(['date','stock'])['fee increase'].sum()/df.groupby(['date','stock'])['trades'].sum()).reset_index()
            fee_increase = fee_increase.rename(columns = {0:'perc_fee_increase'})
            per_stock_fee_increase= fee_increase.groupby(['stock'])['perc_fee_increase'].mean().reset_index()

            fee_merged = relative_old_fee.merge(relative_new_fee, on = ['date','stock'],how = 'outer')
            fee_merged = fee_merged.merge(fee_diff, on = ['date','stock'],how = 'outer')
            fee_merged = fee_merged.merge(relative_fee_diff, on = ['date','stock'],how = 'outer')
            fee_merged = fee_merged.merge(fee_change, on = ['date','stock'],how = 'outer')
            fee_merged = fee_merged.merge(fee_increase, on = ['date','stock'],how = 'outer')
            fee_merged = fee_merged.merge(total_old_fee, on = ['date','stock'],how = 'outer')
            fee_merged = fee_merged.merge(total_new_fee, on = ['date','stock'],how = 'outer')

            df_fee = df[['stock','date','adj mtime','Qualifiers','value','Price','Volume','old fee','relative old fee','new fee','relative new fee','fee diff','relative fee diff','fee change']]

            df_trades = df[df['Price'].notnull()]
            #df.to_csv('C:/Users/anche/NZ/trth data/tester_out/'+'3_'+file_name)
            #df = df[~df['Qualifiers'].isin(['LAM[GV4_TEXT]','LA[GV4_TEXT]','LTM[GV4_TEXT]','PF[GV4_TEXT]','PTM[GV4_TEXT]','SPM[GV4_TEXT]','SP[GV4_TEXT]','OL[GV4_TEXT]','OLM[GV4_TEXT]'])]
            df['trad_direction'] = df['trad_direction'].astype(str)
            df['Price'] = df['Price'].astype(float)
            df['Bid Price'] = df['Bid Price'].astype(float)
            df['Ask Price'] = df['Ask Price'].astype(float)
            df['Volume'] = pd.to_numeric(df['Volume'], errors = 'coerce')
            df['Bid Size abs'] = pd.to_numeric(df['Bid Size abs'],errors = 'coerce')
            df['Ask Size abs'] = pd.to_numeric(df['Ask Size abs'],errors = 'coerce')
            
            
            
            df['issue'] = np.where(((df['trad_direction'] == 'S') & (df['Price'] < df['Bid Price'])) | (
                    (df['trad_direction'] == 'B') & (df['Price'] > df['Ask Price'])), 1, 0)

            df['vol_issue'] = np.where(((df['trad_direction'] == 'S') & (df['Volume'] > df['Bid Size abs'])) | (
                    (df['trad_direction'] == 'B') & (df['Volume'] > df['Ask Size abs'])), 1, 0)

            df['quote_alive'] = df.groupby(['stock','date','venue'])['adj mtime'].shift(-1) -  df['adj mtime']

            df['Quoted Spread'] = np.where(
                (df['Ask Price adj'] != 0) & (df['Bid Price adj'] != 0) & (df['Ask Price adj'] > df['Bid Price adj']),
                df['Ask Price adj'] - df['Bid Price adj'], np.nan)
            df['Quoted Spread bps'] = np.where(
                    (df['Ask Price adj'] != 0) & (df['Bid Price adj'] != 0) & (df['Ask Price adj'] > df['Bid Price adj']),
                    (df['Ask Price adj'] - df['Bid Price adj'])/(df['MidQuote adj']), np.nan)
            df['Quoted Spread_TW'] = df['Quoted Spread'] * df['quote_alive']
            df['Quoted Spread bps_TW'] = df['Quoted Spread bps'] * df['quote_alive']
            
            df['Depth'] = df['Bid Size abs'] + df['Ask Size abs']
            df['Depth_TW'] = df['Depth'] * df['quote_alive']
            
            df['Depth_dollar'] = df['Bid Size abs']*df['Bid Price'] + df['Ask Size abs']*df['Ask Price']
            df['Depth_dollar_TW'] = df['Depth_dollar'] * df['quote_alive']
            

            df[f'at tick'] = np.where(
                        ((df['Quoted Spread'] < 0.01) & (df['MidQuote'] > 0.2)) | (
                            (df['Quoted Spread'] < 0.001) & (df['MidQuote'] < 0.2)), 1, 0)
            df[f'at tick time'] = df['at tick'] * df['quote_alive']
            b_sel = df.trad_direction == 'B'
            df.loc[b_sel, f'Effective Spread'] = 2 * (df.loc[b_sel, f'Price'] -df.loc[b_sel, 'MidQuote adj'])
            s_sel = df.trad_direction == 'S'
            df.loc[s_sel, f'Effective Spread'] = 2 * (df.loc[s_sel, 'MidQuote adj'] -df.loc[s_sel, f'Price'])
            df['pos effective dummy'] = np.where(df['Effective Spread']>0,1,0)
            df[f'Effective Spread_VW'] = df[f'Effective Spread'] * df[f'value']*df['pos effective dummy']
            df.loc[b_sel, f'Effective Spread bps'] = 2 * (df.loc[b_sel, f'Price'] -df.loc[b_sel, 'MidQuote adj'])/df.loc[b_sel, 'MidQuote adj']
            df.loc[s_sel, f'Effective Spread bps'] = 2 * (df.loc[s_sel, 'MidQuote adj'] -df.loc[s_sel, f'Price'])/df.loc[s_sel, 'MidQuote adj']

            df[f'Effective Spread bps_VW'] = df[f'Effective Spread bps'] * df[f'value']*df['pos effective dummy']
            df['value_weight'] = df[f'value']*df['pos effective dummy']
            
            df = df[(df['adj mtime'] > o_time) & (df['adj mtime'] < c_time)]

            trade_value = df.groupby(['date','stock'])['value'].sum().reset_index()
            trade_value = trade_value.rename(columns = {'value':'daily trade value'})

            trade_size = df.groupby(['date','stock'])['value'].mean().reset_index()
            trade_size = trade_size.rename(columns = {'value':'trade size'})

            Quoted_spread_TW = (df.groupby(['stock', 'date'])['Quoted Spread_TW'].sum() / \
                                        df[df['Quoted Spread_TW'].notnull()].groupby(['stock', 'date'])['quote_alive'].sum()).reset_index()
            Quoted_spread_TW = Quoted_spread_TW.rename(columns = {0:'quoted spread'})
            
            Depth_TW = (df.groupby(['stock', 'date'])['Depth_TW'].sum() / \
                                        df[df['Depth_TW'].notnull()].groupby(['stock', 'date'])['quote_alive'].sum()).reset_index()
            Depth_TW = Depth_TW.rename(columns = {0:'Depth'})
            
            Depth_dollar_TW = (df.groupby(['stock', 'date'])['Depth_dollar_TW'].sum() / \
                                        df[df['Depth_TW'].notnull()].groupby(['stock', 'date'])['quote_alive'].sum()).reset_index()
            Depth_dollar_TW = Depth_dollar_TW.rename(columns = {0:'Depth_dollar'})
#             print(df.groupby(['stock', 'date'])['Quoted Spread'].sum())
#             print(Quoted_spread_TW)

            df['MidQuote_TW'] = df['MidQuote adj'] * df['quote_alive']

            midpoint_TW = (df.groupby(['stock', 'date'])['MidQuote_TW'].sum() / \
                                            df[df['MidQuote_TW'].notnull()].groupby(['stock', 'date'])['quote_alive'].sum()).reset_index()
            midpoint_TW = midpoint_TW.rename(columns = {0:'midpoint'})


            Quoted_spread_bps_TW = (df.groupby(['stock', 'date'])['Quoted Spread bps_TW'].sum() / \
                                        df[df['Quoted Spread_TW'].notnull()].groupby(['stock', 'date'])['quote_alive'].sum()).reset_index()
            Quoted_spread_bps_TW = Quoted_spread_bps_TW.rename(columns = {0:'quoted spread bps'})


            min_tick_TW = (df.groupby(['stock', 'date'])['at tick time'].sum() / \
                                  df.groupby(['stock', 'date'])[
                                      'quote_alive'].sum()).reset_index()
            min_tick_TW = min_tick_TW.rename(columns = {0:'tick time'})

            Value_weighted_Effective_Spread = (df.groupby(['stock', 'date'])['Effective Spread_VW'].sum() / \
                                        df[df['Effective Spread_VW'].notnull()].groupby(['stock', 'date'])['value_weight'].sum()).reset_index()

            Value_weighted_Effective_Spread = Value_weighted_Effective_Spread.rename(columns = {0:'value weighted effective spread'})
#             print(Value_weighted_Effective_Spread)
            Value_weighted_Effective_Spread_bps = ((df.groupby(['stock', 'date'])['Effective Spread bps_VW'].sum() / \
                                        df[df['Effective Spread bps_VW'].notnull()].groupby(['stock', 'date'])['value_weight'].sum())*10000).reset_index()
            Value_weighted_Effective_Spread_bps = Value_weighted_Effective_Spread_bps.rename(columns = {0:'value weighted effective spread bps'})

            df = df.sort_values(['adj mtime','mtime_1s','mtime_30s','mtime_1m','mtime_5m'],ascending = True)

            df_realized = pd.merge_asof(df,df[['stock','date','adj mtime','MidQuote adj']],left_on=['mtime_1s'], right_on=['adj mtime'],
                                                 by=['stock','date'], suffixes=('', '_1s_matched'),
                                                 allow_exact_matches=True)
            df_realized = pd.merge_asof(df_realized,df[['stock','date','adj mtime','MidQuote adj']],left_on=['mtime_30s'], right_on=['adj mtime'],
                                                 by=['stock','date'], suffixes=('', '_30s_matched'),
                                                 allow_exact_matches=True)
            df_realized = pd.merge_asof(df_realized,df[['stock','date','adj mtime','MidQuote adj']],left_on=['mtime_1m'], right_on=['adj mtime'],
                                                 by=['stock','date'], suffixes=('', '_1m_matched'),
                                                 allow_exact_matches=True)
            df_realized = pd.merge_asof(df_realized,df[['stock','date','adj mtime','MidQuote adj']],left_on=['mtime_5m'], right_on=['adj mtime'],
                                                 by=['stock','date'], suffixes=('', '_5m_matched'),
                                                 allow_exact_matches=True)
            #1s
            b_sel = df_realized.trad_direction == 'B'
            df_realized.loc[b_sel, f'Realized Spread 1s'] = 2 * (df_realized.loc[b_sel, f'Price'] -df_realized.loc[b_sel, 'MidQuote adj_1s_matched'])
            s_sel = df_realized.trad_direction == 'S'
            df_realized.loc[s_sel, f'Realized Spread 1s'] = 2 * (df_realized.loc[s_sel, 'MidQuote adj_1s_matched'] -df_realized.loc[s_sel, f'Price'])

            df_realized[f'Realized Spread 1s_VW'] = df_realized[f'Realized Spread 1s'] * df_realized[f'value']*df_realized['pos effective dummy']
            df_realized.loc[b_sel, f'Realized Spread 1s bps'] = 2 * (df_realized.loc[b_sel, f'Price'] -df_realized.loc[b_sel, 'MidQuote adj_1s_matched'])/df_realized.loc[b_sel, 'MidQuote adj_1s_matched']
            df_realized.loc[s_sel, f'Realized Spread 1s bps'] = 2 * (df_realized.loc[s_sel, 'MidQuote adj_1s_matched'] -df_realized.loc[s_sel, f'Price'])/df_realized.loc[s_sel, 'MidQuote adj_1s_matched']

            df_realized[f'Realized Spread 1s bps_VW'] = df_realized[f'Realized Spread 1s bps'] * df_realized[f'value']*df_realized['pos effective dummy']
#             df_spreads = df_realized[df_realized['Price'].notnull()][['stock','date','adj mtime','Qualifiers','value','NZ Price','NZ MidQuote adj','Price','Volume','direction','Bid Price adj','Ask Price adj','NZ Bid Price adj','NZ Ask Price adj','Effective Spread','Realized Spread','Effective Spread bps','Realized Spread bps','MidQuote adj','MidQuote adj_1m_matched','Quoted Spread','Quoted Spread bps']]
           
            Value_weighted_Realized_Spread_1s = (df_realized.groupby(['stock', 'date'])['Realized Spread 1s_VW'].sum() / \
                                        df_realized[df_realized['Realized Spread 1s_VW'].notnull()].groupby(['stock', 'date'])['value_weight'].sum()).reset_index()

            Value_weighted_Realized_Spread_1s = Value_weighted_Realized_Spread_1s.rename(columns = {0:'value weighted realized spread 1s'})

            Value_weighted_Realized_Spread_1s_bps = ((df_realized.groupby(['stock', 'date'])['Realized Spread 1s bps_VW'].sum() / \
                                        df_realized[df_realized['Realized Spread 1s bps_VW'].notnull()].groupby(['stock', 'date'])['value_weight'].sum())*10000).reset_index()
            Value_weighted_Realized_Spread_1s_bps = Value_weighted_Realized_Spread_1s_bps.rename(columns = {0:'value weighted realized spread 1s bps'})

            #30s
            
            df_realized.loc[b_sel, f'Realized Spread 30s'] = 2 * (df_realized.loc[b_sel, f'Price'] -df_realized.loc[b_sel, 'MidQuote adj_30s_matched'])
            
            df_realized.loc[s_sel, f'Realized Spread 30s'] = 2 * (df_realized.loc[s_sel, 'MidQuote adj_30s_matched'] -df_realized.loc[s_sel, f'Price'])

            df_realized[f'Realized Spread 30s_VW'] = df_realized[f'Realized Spread 30s'] * df_realized[f'value']*df_realized['pos effective dummy']
            df_realized.loc[b_sel, f'Realized Spread 30s bps'] = 2 * (df_realized.loc[b_sel, f'Price'] -df_realized.loc[b_sel, 'MidQuote adj_30s_matched'])/df_realized.loc[b_sel, 'MidQuote adj_1s_matched']
            df_realized.loc[s_sel, f'Realized Spread 30s bps'] = 2 * (df_realized.loc[s_sel, 'MidQuote adj_30s_matched'] -df_realized.loc[s_sel, f'Price'])/df_realized.loc[s_sel, 'MidQuote adj_1s_matched']

            df_realized[f'Realized Spread 30s bps_VW'] = df_realized[f'Realized Spread 30s bps'] * df_realized[f'value']*df_realized['pos effective dummy']
           
            Value_weighted_Realized_Spread_30s = (df_realized.groupby(['stock', 'date'])['Realized Spread 30s_VW'].sum() / \
                                        df_realized[df_realized['Realized Spread 30s_VW'].notnull()].groupby(['stock', 'date'])['value_weight'].sum()).reset_index()

            Value_weighted_Realized_Spread_30s = Value_weighted_Realized_Spread_30s.rename(columns = {0:'value weighted realized spread 30s'})

            Value_weighted_Realized_Spread_30s_bps = ((df_realized.groupby(['stock', 'date'])['Realized Spread 30s bps_VW'].sum() / \
                                        df_realized[df_realized['Realized Spread 30s bps_VW'].notnull()].groupby(['stock', 'date'])['value_weight'].sum())*10000).reset_index()
            Value_weighted_Realized_Spread_30s_bps = Value_weighted_Realized_Spread_30s_bps.rename(columns = {0:'value weighted realized spread 30s bps'})

            #1m
            
            df_realized.loc[b_sel, f'Realized Spread 1m'] = 2 * (df_realized.loc[b_sel, f'Price'] -df_realized.loc[b_sel, 'MidQuote adj_1m_matched'])
            
            df_realized.loc[s_sel, f'Realized Spread 1m'] = 2 * (df_realized.loc[s_sel, 'MidQuote adj_1m_matched'] -df_realized.loc[s_sel, f'Price'])

            df_realized[f'Realized Spread 1m_VW'] = df_realized[f'Realized Spread 1m'] * df_realized[f'value']*df_realized['pos effective dummy']
            df_realized.loc[b_sel, f'Realized Spread 1m bps'] = 2 * (df_realized.loc[b_sel, f'Price'] -df_realized.loc[b_sel, 'MidQuote adj_1m_matched'])/df_realized.loc[b_sel, 'MidQuote adj_1m_matched']
            df_realized.loc[s_sel, f'Realized Spread 1m bps'] = 2 * (df_realized.loc[s_sel, 'MidQuote adj_1m_matched'] -df_realized.loc[s_sel, f'Price'])/df_realized.loc[s_sel, 'MidQuote adj_1m_matched']

            df_realized[f'Realized Spread 1m bps_VW'] = df_realized[f'Realized Spread 1m bps'] * df_realized[f'value']*df_realized['pos effective dummy']
           
            Value_weighted_Realized_Spread_1m = (df_realized.groupby(['stock', 'date'])['Realized Spread 1m_VW'].sum() / \
                                        df_realized[df_realized['Realized Spread 1m_VW'].notnull()].groupby(['stock', 'date'])['value_weight'].sum()).reset_index()

            Value_weighted_Realized_Spread_1m = Value_weighted_Realized_Spread_1m.rename(columns = {0:'value weighted realized spread 1m'})

            Value_weighted_Realized_Spread_1m_bps = ((df_realized.groupby(['stock', 'date'])['Realized Spread 1m bps_VW'].sum() / \
                                        df_realized[df_realized['Realized Spread 1m bps_VW'].notnull()].groupby(['stock', 'date'])['value_weight'].sum())*10000).reset_index()
            Value_weighted_Realized_Spread_1m_bps = Value_weighted_Realized_Spread_1m_bps.rename(columns = {0:'value weighted realized spread 1m bps'})

            #5m
        
            df_realized.loc[b_sel, f'Realized Spread 5m'] = 2 * (df_realized.loc[b_sel, f'Price'] -df_realized.loc[b_sel, 'MidQuote adj_5m_matched'])
            
            df_realized.loc[s_sel, f'Realized Spread 5m'] = 2 * (df_realized.loc[s_sel, 'MidQuote adj_5m_matched'] -df_realized.loc[s_sel, f'Price'])

            df_realized[f'Realized Spread 5m_VW'] = df_realized[f'Realized Spread 5m'] * df_realized[f'value']*df_realized['pos effective dummy']
            df_realized.loc[b_sel, f'Realized Spread 5m bps'] = 2 * (df_realized.loc[b_sel, f'Price'] -df_realized.loc[b_sel, 'MidQuote adj_5m_matched'])/df_realized.loc[b_sel, 'MidQuote adj_5m_matched']
            df_realized.loc[s_sel, f'Realized Spread 5m bps'] = 2 * (df_realized.loc[s_sel, 'MidQuote adj_5m_matched'] -df_realized.loc[s_sel, f'Price'])/df_realized.loc[s_sel, 'MidQuote adj_5m_matched']

            df_realized[f'Realized Spread 5m bps_VW'] = df_realized[f'Realized Spread 5m bps'] * df_realized[f'value']*df_realized['pos effective dummy']
           
            Value_weighted_Realized_Spread_5m = (df_realized.groupby(['stock', 'date'])['Realized Spread 5m_VW'].sum() / \
                                        df_realized[df_realized['Realized Spread 5m_VW'].notnull()].groupby(['stock', 'date'])['value_weight'].sum()).reset_index()

            Value_weighted_Realized_Spread_5m = Value_weighted_Realized_Spread_5m.rename(columns = {0:'value weighted realized spread 5m'})

            Value_weighted_Realized_Spread_5m_bps = ((df_realized.groupby(['stock', 'date'])['Realized Spread 5m bps_VW'].sum() / \
                                        df_realized[df_realized['Realized Spread 5m bps_VW'].notnull()].groupby(['stock', 'date'])['value_weight'].sum())*10000).reset_index()
            Value_weighted_Realized_Spread_5m_bps = Value_weighted_Realized_Spread_5m_bps.rename(columns = {0:'value weighted realized spread 5m bps'})
            #df_realized.to_csv('C://Users//anche//NZ//tables update//check//'+df['date'].astype(str).iloc[0]+df['stock'].iloc[0]+'.csv')

            #compute market order 
            #market_order = df.groupby(['stock','date','adj mtime'])['value'].sum().reset_index()

            #daily_market_order = market_order.groupby(['stock','date'])['value'].mean().reset_index()
            #daily_market_order = daily_market_order.rename(columns = {'value':'market order value'})
            
            ##########################for buyer initiated##############################


            df_trades_buy = df.loc[
                (df['direction'] == 'B')]  
            if len(df_trades_buy)>1:
                df_trades_buy = df_trades_buy.reset_index()
                #if the difference in time between each two trades is more than 200, then start a new trade string 
                df_trades_buy['trade_dif'] = df_trades_buy['adj mtime'] - df_trades_buy.groupby(['date','stock'])['adj mtime'].shift()
                
                df_trades_buy['trade_string_index'] = int()

                df_trades_buy['trade_string_index'] = (df_trades_buy['trade_dif'] > 200000).cumsum()
                
                buy_string_market_order = df_trades_buy.groupby(['date','stock','trade_string_index'])['value'].sum().rename('market_order').reset_index()
                buy_string_limit_order = df_trades_buy.groupby(['date','stock','trade_string_index'])['value'].apply(lambda x: x.iloc[-1:].mean()).rename('limit_order').reset_index()

                buy_market_order = buy_string_market_order.groupby(['date','stock'])['market_order'].mean().rename('buy_market_order').reset_index()
                buy_limit_order = buy_string_limit_order.groupby(['date','stock'])['limit_order'].mean().rename('buy_limit_order').reset_index()
                
                #print(buy_limit_order)
            
            df_trades_sell = df.loc[
                (df['direction'] == 'S')]  
            if len(df_trades_sell)>1:
                df_trades_sell = df_trades_sell.reset_index()
                df_trades_sell['trade_dif'] = df_trades_sell['adj mtime'] - df_trades_sell.groupby(['date','stock'])['adj mtime'].shift()
                
                df_trades_sell['trade_string_index'] = int()

                df_trades_sell['trade_string_index'] = (df_trades_sell['trade_dif'] > 200000).cumsum()
                sell_string_market_order = df_trades_sell.groupby(['date','stock','trade_string_index'])['value'].sum().rename('market_order').reset_index()
                sell_string_limit_order = df_trades_sell.groupby(['date','stock','trade_string_index'])['value'].apply(lambda x: x.iloc[-1:].mean()).rename('limit_order').reset_index()
                
                sell_market_order = sell_string_market_order.groupby(['date','stock'])['market_order'].mean().rename('sell_market_order').reset_index()
                sell_limit_order = sell_string_limit_order.groupby(['date','stock'])['limit_order'].mean().rename('sell_limit_order').reset_index()
                #print(sell_limit_order)
            #all_type_orders = buy_market_order.merge(buy_limit_order, on = ['date','stock']).merge(sell_market_order, on = ['date','stock']).merge(sell_limit_order, on = ['date','stock'])
            
            '''
            #compute limit order

            df['bid diff'] = df.groupby(['stock','venue','date'])['Bid Price'].diff() 
            df['bid order'] = np.where(df['bid diff']==0,df.groupby(['stock','venue','date'])['Bid Size abs'].diff(),np.nan )
            df['bid order'] = np.where(df['bid diff']>0,df['Bid Size abs'],df['bid order'])

            df['bid order'] = np.where(df['bid order']>0,df['bid order'],np.nan)

            df['ask diff'] = df.groupby(['stock','venue','date'])['Ask Price'].diff() 
            df['ask order'] = np.where(df['ask diff']==0,df.groupby(['stock','venue','date'])['Ask Size abs'].diff(),np.nan )
            df['ask order'] = np.where(df['ask diff']<0,df['Ask Size abs'],df['ask order'])
            df['ask order'] = np.where(df['ask order']>0,df['ask order'],np.nan)

            daily_bid_order = df.groupby(['stock','date'])['bid order'].mean().reset_index()
            daily_bid_order = daily_bid_order.rename(columns = {'bid order':'bid order value'})

            daily_ask_order = df.groupby(['stock','date'])['ask order'].mean().reset_index()
            daily_ask_order = daily_ask_order.rename(columns = {'ask order':'ask order value'})

            daily_order = daily_bid_order.merge(daily_ask_order, on = ['stock','date'])

            '''
            num_quote_update = df.groupby(['stock','date'])['Bid Price'].count().reset_index()
            num_quote_update = num_quote_update.rename(columns = {'Bid Price':'num_quote_update'})

            num_trades = df.groupby(['stock','date'])['Price'].count().reset_index()
            num_trades = num_trades.rename(columns = {'Price':'num_trades'})

            fee_diff['date'] = pd.to_datetime(fee_diff['date'])
            fee_change['date'] = pd.to_datetime(fee_change['date'])
            fee_increase['date'] = pd.to_datetime(fee_increase['date'])
            # daily_value['date'] = pd.to_datetime(daily_value['date'])
            trade_value['date'] = pd.to_datetime(trade_value['date'])
            Quoted_spread_TW['date'] = pd.to_datetime(Quoted_spread_TW['date'])
            Depth_TW['date'] = pd.to_datetime(Depth_TW['date'])
            Depth_dollar_TW['date'] = pd.to_datetime(Depth_dollar_TW['date'])
            min_tick_TW['date'] = pd.to_datetime(min_tick_TW['date'])
            Value_weighted_Effective_Spread['date'] = pd.to_datetime(Value_weighted_Effective_Spread['date'])
            Value_weighted_Effective_Spread_bps['date'] = pd.to_datetime(Value_weighted_Effective_Spread_bps['date'])
            Value_weighted_Realized_Spread_1s['date'] = pd.to_datetime(Value_weighted_Realized_Spread_1s['date'])
            Value_weighted_Realized_Spread_1s_bps['date'] = pd.to_datetime(Value_weighted_Realized_Spread_1s_bps['date'])
            
            Value_weighted_Realized_Spread_30s['date'] = pd.to_datetime(Value_weighted_Realized_Spread_30s['date'])
            Value_weighted_Realized_Spread_30s_bps['date'] = pd.to_datetime(Value_weighted_Realized_Spread_30s_bps['date'])
            Value_weighted_Realized_Spread_1m['date'] = pd.to_datetime(Value_weighted_Realized_Spread_1m['date'])
            Value_weighted_Realized_Spread_1m_bps['date'] = pd.to_datetime(Value_weighted_Realized_Spread_1m_bps['date'])
            Value_weighted_Realized_Spread_5m['date'] = pd.to_datetime(Value_weighted_Realized_Spread_5m['date'])
            Value_weighted_Realized_Spread_5m_bps['date'] = pd.to_datetime(Value_weighted_Realized_Spread_5m_bps['date'])

            
            
            #             daily_market_order['date'] = pd.to_datetime(daily_market_order['date'])
#             daily_order['date'] = pd.to_datetime(daily_order['date'])
            num_quote_update['date'] = pd.to_datetime(num_quote_update['date'])
            num_trades['date'] = pd.to_datetime(num_trades['date'])
            midpoint_TW['date'] = pd.to_datetime(midpoint_TW['date'])
            
            trade_size['date'] = pd.to_datetime(trade_size['date'])
            try:
                buy_market_order['date'] = pd.to_datetime(buy_market_order['date'])
                buy_limit_order['date'] = pd.to_datetime(buy_limit_order['date'])
            except:
                print(filename)
            try:    
                sell_market_order['date'] = pd.to_datetime(sell_market_order['date'])
                sell_limit_order['date'] = pd.to_datetime(sell_limit_order['date'])

            except:
                print(filename)
            all_metric = fee_diff.merge(fee_change, on = ['stock','date'], how = 'outer').merge(fee_increase, on = ['stock','date'], how = 'outer')
            all_metric = all_metric.merge(trade_value, on = ['stock','date'], how = 'outer')
            all_metric = all_metric.merge(Quoted_spread_TW,on = ['stock','date'], how = 'outer')
            all_metric = all_metric.merge(Depth_TW,on = ['stock','date'], how = 'outer')
            all_metric = all_metric.merge(Depth_dollar_TW,on = ['stock','date'], how = 'outer')
            all_metric = all_metric.merge(min_tick_TW,on = ['stock','date'], how = 'outer')
            all_metric = all_metric.merge(Value_weighted_Effective_Spread,on = ['stock','date'], how = 'outer')
            all_metric = all_metric.merge(Value_weighted_Effective_Spread_bps,on = ['stock','date'], how = 'outer')
            all_metric = all_metric.merge(Value_weighted_Realized_Spread_1s,on = ['stock','date'], how = 'outer')
            all_metric = all_metric.merge(Value_weighted_Realized_Spread_1s_bps,on = ['stock','date'], how = 'outer')
            all_metric = all_metric.merge(Value_weighted_Realized_Spread_30s,on = ['stock','date'], how = 'outer')
            all_metric = all_metric.merge(Value_weighted_Realized_Spread_30s_bps,on = ['stock','date'], how = 'outer')
            all_metric = all_metric.merge(Value_weighted_Realized_Spread_1m,on = ['stock','date'], how = 'outer')
            all_metric = all_metric.merge(Value_weighted_Realized_Spread_1m_bps,on = ['stock','date'], how = 'outer')
            all_metric = all_metric.merge(Value_weighted_Realized_Spread_5m,on = ['stock','date'], how = 'outer')
            all_metric = all_metric.merge(Value_weighted_Realized_Spread_5m_bps,on = ['stock','date'], how = 'outer')
#             all_metric = all_metric.merge(daily_market_order,on = ['stock','date'], how = 'outer')
#             all_metric = all_metric.merge(daily_order,on = ['stock','date'], how = 'outer')
            all_metric = all_metric.merge(num_quote_update,on = ['stock','date'], how = 'outer')
            all_metric = all_metric.merge(num_trades,on = ['stock','date'], how = 'outer')
            all_metric = all_metric.merge(midpoint_TW,on = ['stock','date'], how = 'outer')
            all_metric = all_metric.merge(trade_size,on = ['stock','date'], how = 'outer')
            try:
                all_metric = all_metric.merge(buy_market_order,on = ['stock','date'], how = 'outer')
                all_metric = all_metric.merge(buy_limit_order,on = ['stock','date'], how = 'outer')
            except:
                print(filename)
            try:
                all_metric = all_metric.merge(sell_market_order,on = ['stock','date'], how = 'outer')
                all_metric = all_metric.merge(sell_limit_order,on = ['stock','date'], how = 'outer')
            except:
                print(filename)
            
            return (all_metric,df_trades,fee_merged,df_fee)




In [6]:
import os
os.chdir('C:/Users/anche/NZ/trth data/NZALL TAQ')
metric_list = []
trades_list = []
fee_merged_list = []
fee_list = []
spreads_list = []
for file in os.listdir('C:/Users/anche/NZ/trth data/NZALL TAQ'):
    try:
#         if 'TWR.NZ' in file:
        
            metric_list.append(metric(file)[0])
            trades_list.append(metric(file)[1])
            #fee_merged_list.append(metric(file)[2])
            #fee_list.append(metric(file)[3])
            #spreads_list.append(metric(file)[4])
            #metric(file)[4].to_csv('C:/Users/anche/NZ/tables v2/NZ new/stock days whole/'+file.split('/')[-1])
    except:
        print(file,'something')

TAQ_2018-09-03_AFT.NZ.csv
TAQ_2018-09-03_AFT.NZ.csv
TAQ_2018-09-03_AFT.NZ.csv
TAQ_2018-09-03_AFT.NZ.csv
TAQ_2018-09-03_AFT.NZ.csv
TAQ_2018-09-03_AFT.NZ.csv
TAQ_2018-09-03_AFT.NZ.csv
TAQ_2018-09-03_AFT.NZ.csv
TAQ_2018-09-03_ALF.NZ.csv
TAQ_2018-09-03_ALF.NZ.csv
TAQ_2018-09-03_ALF.NZ.csv
TAQ_2018-09-03_ALF.NZ.csv
TAQ_2018-09-03_ALF.NZ.csv
TAQ_2018-09-03_ALF.NZ.csv
TAQ_2018-09-03_ALF.NZ.csv
TAQ_2018-09-03_ALF.NZ.csv
TAQ_2018-09-03_AOR.NZ.csv something
TAQ_2018-09-03_APL.NZ.csv
TAQ_2018-09-03_APL.NZ.csv
TAQ_2018-09-03_APL.NZ.csv
TAQ_2018-09-03_APL.NZ.csv
TAQ_2018-09-03_AUG.NZ.csv
TAQ_2018-09-03_AUG.NZ.csv
TAQ_2018-09-03_AUG.NZ.csv
TAQ_2018-09-03_AUG.NZ.csv
TAQ_2018-09-03_AWF.NZ.csv
TAQ_2018-09-03_AWF.NZ.csv
TAQ_2018-09-03_AWF.NZ.csv
TAQ_2018-09-03_AWF.NZ.csv
TAQ_2018-09-03_AWF.NZ.csv
TAQ_2018-09-03_AWF.NZ.csv
TAQ_2018-09-03_AWF.NZ.csv
TAQ_2018-09-03_AWF.NZ.csv
TAQ_2018-09-03_BGP.NZ.csv
TAQ_2018-09-03_BGP.NZ.csv
TAQ_2018-09-03_BGP.NZ.csv
TAQ_2018-09-03_BGP.NZ.csv
TAQ_2018-09-03_BLT.NZ.csv
TA

TAQ_2018-09-04_CMO.NZ.csv
TAQ_2018-09-04_CMO.NZ.csv
TAQ_2018-09-04_CMO.NZ.csv
TAQ_2018-09-04_CMO.NZ.csv
TAQ_2018-09-04_CVT.NZ.csv
TAQ_2018-09-04_CVT.NZ.csv
TAQ_2018-09-04_CVT.NZ.csv
TAQ_2018-09-04_CVT.NZ.csv
TAQ_2018-09-04_ERD.NZ.csv
TAQ_2018-09-04_ERD.NZ.csv
TAQ_2018-09-04_ERD.NZ.csv
TAQ_2018-09-04_ERD.NZ.csv
TAQ_2018-09-04_ERD.NZ.csv
TAQ_2018-09-04_ERD.NZ.csv
TAQ_2018-09-04_ERD.NZ.csv
TAQ_2018-09-04_ERD.NZ.csv
TAQ_2018-09-04_FIN.NZ.csv something
TAQ_2018-09-04_FMS.NZ.csv
TAQ_2018-09-04_FMS.NZ.csv
TAQ_2018-09-04_FMS.NZ.csv
TAQ_2018-09-04_FMS.NZ.csv
TAQ_2018-09-04_FMS.NZ.csv
TAQ_2018-09-04_FMS.NZ.csv
TAQ_2018-09-04_FMS.NZ.csv
TAQ_2018-09-04_FMS.NZ.csv
TAQ_2018-09-04_GEO.NZ.csv
TAQ_2018-09-04_GEO.NZ.csv
TAQ_2018-09-04_GEO.NZ.csv
TAQ_2018-09-04_GEO.NZ.csv
TAQ_2018-09-04_GEO.NZ.csv
TAQ_2018-09-04_GEO.NZ.csv
TAQ_2018-09-04_GEO.NZ.csv
TAQ_2018-09-04_GEO.NZ.csv
TAQ_2018-09-04_GXH.NZ.csv
TAQ_2018-09-04_GXH.NZ.csv
TAQ_2018-09-04_GXH.NZ.csv
TAQ_2018-09-04_GXH.NZ.csv
TAQ_2018-09-04_IKE.NZ.csv
TA

TAQ_2018-09-05_GEO.NZ.csv
TAQ_2018-09-05_GEO.NZ.csv
TAQ_2018-09-05_GEO.NZ.csv
TAQ_2018-09-05_GEO.NZ.csv
TAQ_2018-09-05_GXH.NZ.csv
TAQ_2018-09-05_GXH.NZ.csv
TAQ_2018-09-05_GXH.NZ.csv
TAQ_2018-09-05_GXH.NZ.csv
TAQ_2018-09-05_GXH.NZ.csv
TAQ_2018-09-05_GXH.NZ.csv
TAQ_2018-09-05_GXH.NZ.csv
TAQ_2018-09-05_GXH.NZ.csv
TAQ_2018-09-05_IPL.NZ.csv
TAQ_2018-09-05_IPL.NZ.csv
TAQ_2018-09-05_IPL.NZ.csv
TAQ_2018-09-05_IPL.NZ.csv
TAQ_2018-09-05_KMD.NZ.csv
TAQ_2018-09-05_KMD.NZ.csv
TAQ_2018-09-05_KMD.NZ.csv
TAQ_2018-09-05_KMD.NZ.csv
TAQ_2018-09-05_MAD.NZ.csv
TAQ_2018-09-05_MAD.NZ.csv
TAQ_2018-09-05_MAD.NZ.csv
TAQ_2018-09-05_MAD.NZ.csv
TAQ_2018-09-05_MAD.NZ.csv
TAQ_2018-09-05_MAD.NZ.csv
TAQ_2018-09-05_MAD.NZ.csv
TAQ_2018-09-05_MAD.NZ.csv
TAQ_2018-09-05_MCK.NZ.csv
TAQ_2018-09-05_MCK.NZ.csv
TAQ_2018-09-05_MCK.NZ.csv
TAQ_2018-09-05_MCK.NZ.csv
TAQ_2018-09-05_MCK.NZ.csv
TAQ_2018-09-05_MCK.NZ.csv
TAQ_2018-09-05_MCK.NZ.csv
TAQ_2018-09-05_MCK.NZ.csv
TAQ_2018-09-05_MGL.NZ.csv
TAQ_2018-09-05_MGL.NZ.csv
TAQ_2018-09-

TAQ_2018-09-06_MAD.NZ.csv
TAQ_2018-09-06_MAD.NZ.csv
TAQ_2018-09-06_MAD.NZ.csv
TAQ_2018-09-06_MAD.NZ.csv
TAQ_2018-09-06_MGL.NZ.csv
TAQ_2018-09-06_MGL.NZ.csv
TAQ_2018-09-06_MGL.NZ.csv
TAQ_2018-09-06_MGL.NZ.csv
TAQ_2018-09-06_MMH.NZ.csv
TAQ_2018-09-06_MMH.NZ.csv
TAQ_2018-09-06_MMH.NZ.csv
TAQ_2018-09-06_MMH.NZ.csv
TAQ_2018-09-06_MOA.NZ.csv
TAQ_2018-09-06_MOA.NZ.csv
TAQ_2018-09-06_MOA.NZ.csv
TAQ_2018-09-06_MOA.NZ.csv
TAQ_2018-09-06_MOA.NZ.csv
TAQ_2018-09-06_MOA.NZ.csv
TAQ_2018-09-06_MOA.NZ.csv
TAQ_2018-09-06_MOA.NZ.csv
TAQ_2018-09-06_NTL.NZ.csv
TAQ_2018-09-06_NTL.NZ.csv
TAQ_2018-09-06_NTL.NZ.csv
TAQ_2018-09-06_NTL.NZ.csv
TAQ_2018-09-06_NWF.NZ.csv
TAQ_2018-09-06_NWF.NZ.csv
TAQ_2018-09-06_NWF.NZ.csv
TAQ_2018-09-06_NWF.NZ.csv
TAQ_2018-09-06_NZO.NZ.csv
TAQ_2018-09-06_NZO.NZ.csv
TAQ_2018-09-06_NZO.NZ.csv
TAQ_2018-09-06_NZO.NZ.csv
TAQ_2018-09-06_PGC.NZ.csv
TAQ_2018-09-06_PGC.NZ.csv
TAQ_2018-09-06_PGC.NZ.csv
TAQ_2018-09-06_PGC.NZ.csv
TAQ_2018-09-06_PGC.NZ.csv
TAQ_2018-09-06_PGC.NZ.csv
TAQ_2018-09-

TAQ_2018-09-07_MPG.NZ.csv
TAQ_2018-09-07_MPG.NZ.csv
TAQ_2018-09-07_MVN.NZ.csv
TAQ_2018-09-07_MVN.NZ.csv
TAQ_2018-09-07_MVN.NZ.csv
TAQ_2018-09-07_MVN.NZ.csv
TAQ_2018-09-07_NWF.NZ.csv
TAQ_2018-09-07_NWF.NZ.csv
TAQ_2018-09-07_NWF.NZ.csv
TAQ_2018-09-07_NWF.NZ.csv
TAQ_2018-09-07_NWF.NZ.csv
TAQ_2018-09-07_NWF.NZ.csv
TAQ_2018-09-07_NWF.NZ.csv
TAQ_2018-09-07_NWF.NZ.csv
TAQ_2018-09-07_NZO.NZ.csv
TAQ_2018-09-07_NZO.NZ.csv
TAQ_2018-09-07_NZO.NZ.csv
TAQ_2018-09-07_NZO.NZ.csv
TAQ_2018-09-07_PGC.NZ.csv
TAQ_2018-09-07_PGC.NZ.csv
TAQ_2018-09-07_PGC.NZ.csv
TAQ_2018-09-07_PGC.NZ.csv
TAQ_2018-09-07_PGC.NZ.csv
TAQ_2018-09-07_PGC.NZ.csv
TAQ_2018-09-07_PGC.NZ.csv
TAQ_2018-09-07_PGC.NZ.csv
TAQ_2018-09-07_PGW.NZ.csv
TAQ_2018-09-07_PGW.NZ.csv
TAQ_2018-09-07_PGW.NZ.csv
TAQ_2018-09-07_PGW.NZ.csv
TAQ_2018-09-07_PIL.NZ.csv
TAQ_2018-09-07_PIL.NZ.csv
TAQ_2018-09-07_PIL.NZ.csv
TAQ_2018-09-07_PIL.NZ.csv
TAQ_2018-09-07_PIL.NZ.csv
TAQ_2018-09-07_PIL.NZ.csv
TAQ_2018-09-07_PIL.NZ.csv
TAQ_2018-09-07_PIL.NZ.csv
TAQ_2018-09-

...

TAQ_2018-11-26_GEO.NZ.csv
TAQ_2018-11-26_GEO.NZ.csv
TAQ_2018-11-26_GEO.NZ.csv
TAQ_2018-11-26_GEO.NZ.csv
TAQ_2018-11-26_GEO.NZ.csv
TAQ_2018-11-26_GEO.NZ.csv
TAQ_2018-11-26_GEO.NZ.csv
TAQ_2018-11-26_GEO.NZ.csv
TAQ_2018-11-26_GTK.NZ.csv
TAQ_2018-11-26_GTK.NZ.csv
TAQ_2018-11-26_GTK.NZ.csv
TAQ_2018-11-26_GTK.NZ.csv
TAQ_2018-11-26_IKE.NZ.csv
TAQ_2018-11-26_IKE.NZ.csv
TAQ_2018-11-26_IKE.NZ.csv
TAQ_2018-11-26_IKE.NZ.csv
TAQ_2018-11-26_IKE.NZ.csv
TAQ_2018-11-26_IKE.NZ.csv
TAQ_2018-11-26_IKE.NZ.csv
TAQ_2018-11-26_IKE.NZ.csv
TAQ_2018-11-26_IPL.NZ.csv
TAQ_2018-11-26_IPL.NZ.csv
TAQ_2018-11-26_IPL.NZ.csv
TAQ_2018-11-26_IPL.NZ.csv
TAQ_2018-11-26_MAD.NZ.csv
TAQ_2018-11-26_MAD.NZ.csv
TAQ_2018-11-26_MAD.NZ.csv
TAQ_2018-11-26_MAD.NZ.csv
TAQ_2018-11-26_MAD.NZ.csv
TAQ_2018-11-26_MAD.NZ.csv
TAQ_2018-11-26_MAD.NZ.csv
TAQ_2018-11-26_MAD.NZ.csv
TAQ_2018-11-26_MCK.NZ.csv
TAQ_2018-11-26_MCK.NZ.csv
TAQ_2018-11-26_MCK.NZ.csv
TAQ_2018-11-26_MCK.NZ.csv
TAQ_2018-11-26_MCK.NZ.csv
TAQ_2018-11-26_MCK.NZ.csv
TAQ_2018-11-

TAQ_2018-11-27_FMS.NZ.csv
TAQ_2018-11-27_FMS.NZ.csv
TAQ_2018-11-27_FMS.NZ.csv
TAQ_2018-11-27_FMS.NZ.csv
TAQ_2018-11-27_GEO.NZ.csv
TAQ_2018-11-27_GEO.NZ.csv
TAQ_2018-11-27_GEO.NZ.csv
TAQ_2018-11-27_GEO.NZ.csv
TAQ_2018-11-27_GEO.NZ.csv
TAQ_2018-11-27_GEO.NZ.csv
TAQ_2018-11-27_GEO.NZ.csv
TAQ_2018-11-27_GEO.NZ.csv
TAQ_2018-11-27_GXH.NZ.csv
TAQ_2018-11-27_GXH.NZ.csv
TAQ_2018-11-27_GXH.NZ.csv
TAQ_2018-11-27_GXH.NZ.csv
TAQ_2018-11-27_HGH.NZ.csv
TAQ_2018-11-27_HGH.NZ.csv
TAQ_2018-11-27_HGH.NZ.csv
TAQ_2018-11-27_HGH.NZ.csv
TAQ_2018-11-27_IKE.NZ.csv
TAQ_2018-11-27_IKE.NZ.csv
TAQ_2018-11-27_IKE.NZ.csv
TAQ_2018-11-27_IKE.NZ.csv
TAQ_2018-11-27_IKE.NZ.csv
TAQ_2018-11-27_IKE.NZ.csv
TAQ_2018-11-27_IKE.NZ.csv
TAQ_2018-11-27_IKE.NZ.csv
TAQ_2018-11-27_MAD.NZ.csv
TAQ_2018-11-27_MAD.NZ.csv
TAQ_2018-11-27_MAD.NZ.csv
TAQ_2018-11-27_MAD.NZ.csv
TAQ_2018-11-27_MAD.NZ.csv
TAQ_2018-11-27_MAD.NZ.csv
TAQ_2018-11-27_MAD.NZ.csv
TAQ_2018-11-27_MAD.NZ.csv
TAQ_2018-11-27_MCK.NZ.csv
TAQ_2018-11-27_MCK.NZ.csv
TAQ_2018-11-

TAQ_2018-11-28_FMS.NZ.csv
TAQ_2018-11-28_FMS.NZ.csv
TAQ_2018-11-28_FMS.NZ.csv
TAQ_2018-11-28_FMS.NZ.csv
TAQ_2018-11-28_GEO.NZ.csv
TAQ_2018-11-28_GEO.NZ.csv
TAQ_2018-11-28_GEO.NZ.csv
TAQ_2018-11-28_GEO.NZ.csv
TAQ_2018-11-28_GEO.NZ.csv
TAQ_2018-11-28_GEO.NZ.csv
TAQ_2018-11-28_GEO.NZ.csv
TAQ_2018-11-28_GEO.NZ.csv
TAQ_2018-11-28_GTK.NZ.csv
TAQ_2018-11-28_GTK.NZ.csv
TAQ_2018-11-28_GTK.NZ.csv
TAQ_2018-11-28_GTK.NZ.csv
TAQ_2018-11-28_IKE.NZ.csv
TAQ_2018-11-28_IKE.NZ.csv
TAQ_2018-11-28_IKE.NZ.csv
TAQ_2018-11-28_IKE.NZ.csv
TAQ_2018-11-28_MAD.NZ.csv
TAQ_2018-11-28_MAD.NZ.csv
TAQ_2018-11-28_MAD.NZ.csv
TAQ_2018-11-28_MAD.NZ.csv
TAQ_2018-11-28_MAD.NZ.csv
TAQ_2018-11-28_MAD.NZ.csv
TAQ_2018-11-28_MAD.NZ.csv
TAQ_2018-11-28_MAD.NZ.csv
TAQ_2018-11-28_MCK.NZ.csv
TAQ_2018-11-28_MCK.NZ.csv
TAQ_2018-11-28_MCK.NZ.csv
TAQ_2018-11-28_MCK.NZ.csv
TAQ_2018-11-28_MCK.NZ.csv
TAQ_2018-11-28_MCK.NZ.csv
TAQ_2018-11-28_MCK.NZ.csv
TAQ_2018-11-28_MCK.NZ.csv
TAQ_2018-11-28_MGL.NZ.csv
TAQ_2018-11-28_MGL.NZ.csv
TAQ_2018-11-

TAQ_2018-11-29_CMO.NZ.csv
TAQ_2018-11-29_CMO.NZ.csv
TAQ_2018-11-29_CMO.NZ.csv
TAQ_2018-11-29_CMO.NZ.csv
TAQ_2018-11-29_CVT.NZ.csv
TAQ_2018-11-29_CVT.NZ.csv
TAQ_2018-11-29_CVT.NZ.csv
TAQ_2018-11-29_CVT.NZ.csv
TAQ_2018-11-29_DGL.NZ.csv
TAQ_2018-11-29_DGL.NZ.csv
TAQ_2018-11-29_DGL.NZ.csv
TAQ_2018-11-29_DGL.NZ.csv
TAQ_2018-11-29_ERD.NZ.csv
TAQ_2018-11-29_ERD.NZ.csv
TAQ_2018-11-29_ERD.NZ.csv
TAQ_2018-11-29_ERD.NZ.csv
TAQ_2018-11-29_ERD.NZ.csv
TAQ_2018-11-29_ERD.NZ.csv
TAQ_2018-11-29_ERD.NZ.csv
TAQ_2018-11-29_ERD.NZ.csv
TAQ_2018-11-29_FIN.NZ.csv something
TAQ_2018-11-29_FMS.NZ.csv
TAQ_2018-11-29_FMS.NZ.csv
TAQ_2018-11-29_FMS.NZ.csv
TAQ_2018-11-29_FMS.NZ.csv
TAQ_2018-11-29_FMS.NZ.csv
TAQ_2018-11-29_FMS.NZ.csv
TAQ_2018-11-29_FMS.NZ.csv
TAQ_2018-11-29_FMS.NZ.csv
TAQ_2018-11-29_GEO.NZ.csv
TAQ_2018-11-29_GEO.NZ.csv
TAQ_2018-11-29_GEO.NZ.csv
TAQ_2018-11-29_GEO.NZ.csv
TAQ_2018-11-29_GEO.NZ.csv
TAQ_2018-11-29_GEO.NZ.csv
TAQ_2018-11-29_GEO.NZ.csv
TAQ_2018-11-29_GEO.NZ.csv
TAQ_2018-11-29_GXH.NZ.csv
TA

TAQ_2018-11-30_FMS.NZ.csv
TAQ_2018-11-30_FMS.NZ.csv
TAQ_2018-11-30_FMS.NZ.csv
TAQ_2018-11-30_FMS.NZ.csv
TAQ_2018-11-30_GEO.NZ.csv
TAQ_2018-11-30_GEO.NZ.csv
TAQ_2018-11-30_GEO.NZ.csv
TAQ_2018-11-30_GEO.NZ.csv
TAQ_2018-11-30_GEO.NZ.csv
TAQ_2018-11-30_GEO.NZ.csv
TAQ_2018-11-30_GEO.NZ.csv
TAQ_2018-11-30_GEO.NZ.csv
TAQ_2018-11-30_GXH.NZ.csv
TAQ_2018-11-30_GXH.NZ.csv
TAQ_2018-11-30_GXH.NZ.csv
TAQ_2018-11-30_GXH.NZ.csv
TAQ_2018-11-30_IKE.NZ.csv
TAQ_2018-11-30_IKE.NZ.csv
TAQ_2018-11-30_IKE.NZ.csv
TAQ_2018-11-30_IKE.NZ.csv
TAQ_2018-11-30_IKE.NZ.csv
TAQ_2018-11-30_IKE.NZ.csv
TAQ_2018-11-30_IKE.NZ.csv
TAQ_2018-11-30_IKE.NZ.csv
TAQ_2018-11-30_MAD.NZ.csv
TAQ_2018-11-30_MAD.NZ.csv
TAQ_2018-11-30_MAD.NZ.csv
TAQ_2018-11-30_MAD.NZ.csv
TAQ_2018-11-30_MAD.NZ.csv
TAQ_2018-11-30_MAD.NZ.csv
TAQ_2018-11-30_MAD.NZ.csv
TAQ_2018-11-30_MAD.NZ.csv
TAQ_2018-11-30_MCK.NZ.csv
TAQ_2018-11-30_MCK.NZ.csv
TAQ_2018-11-30_MCK.NZ.csv
TAQ_2018-11-30_MCK.NZ.csv
TAQ_2018-11-30_MCK.NZ.csv
TAQ_2018-11-30_MCK.NZ.csv
TAQ_2018-11-

In [7]:
all_metrics = pd.concat(metric_list)
all_metrics.to_csv('C:/Users/anche/NZ/tables update/NZ metric allord v9.csv')

C:\Users\anche\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [8]:
all_trades = pd.concat(trades_list)
all_trades.to_csv('C:/Users/anche/NZ/tables update/NZ trades allord v9.csv')

In [ ]:
all_metrics = pd.concat(metric_list)
all_metrics.to_csv('C://Users//anche//NZ//tables update//check//TWR.csv')

In [ ]:
# all_trades = pd.concat(trades_list)
# all_trades.to_csv('C://Users//anche//NZ//tables update//check//AIRNZ.csv')

In [ ]:
fee_list

In [ ]:
first

In [ ]:
all_fee_merged = pd.concat(fee_merged_list)
all_fee_merged.to_csv('C:/Users/anche/NZ/tables v2/NZ all_fee_merged allord v9.csv')

In [ ]:
all_fee = pd.concat(fee_list)
all_fee.to_csv('C:/Users/anche/NZ/tables v2/NZ all_fee allord v9.csv')

In [ ]:
all_spreads = pd.concat(spreads_list)
all_spreads.to_csv('C:/Users/anche/NZ/tables v2/NZ all_spreads allord v9.csv')